<a href="https://colab.research.google.com/github/Shona173/codes/blob/main/SDF_3D_Shape_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install POT

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import ot

In [ ]:
def ndot(a,b):
    return a[0]*b[0]-a[1]*b[1]

In [ ]:
def clamp(value, min_val, max_val):
    return max(min_val, min(value, max_val))

In [ ]:
def sdf_sphere(x, r=0.5):
    return np.sqrt(np.sum(x**2, axis=1)) - r

In [ ]:
def sdf_box(p, b=[0.7,0.7,0.7]):
    q = np.abs(p) - b
    return np.linalg.norm(np.maximum(q, 0.0)) + min(max(q[:,0], max(q[:,1], q[:,2])), 0.0)

In [ ]:
def sdf_cylinder(p, c=[0.7,0.7,0.7]):
    return np.linalg.norm(p[[:,0],[:,2]] - c[[:,0],[:,1]]) - c[:,2]

In [ ]:
def sdf_tri_prism(p, h=[0.7,0.7]):
    q = np.abs(p)
    return max(q[:,2] - h[:,1], max(q[:,0] * 0.866025 + p[:,1] * 0.5, -p[:,1]) - h[:,0] * 0.5)

In [ ]:
def sdf_capped_cone(p, h=0.5, r1=0.5, r2=0.5):
    q = np.array([np.linalg.norm(p[:2]), p[:,2]])
    k1 = np.array([r2, h])
    k2 = np.array([r2 - r1, 2.0 * h])

    ca = np.array([q[:,0] - min(q[:,0], r1 if q[:,1] < 0.0 else r2), abs(q[:,1]) - h])

    dot_k1_q = np.dot(k1 - q, k2)
    dot_k2 = dot2(k2)
    cb = q - k1 + k2 * np.clip(dot_k1_q / dot_k2, 0.0, 1.0)

    s = -1.0 if (cb[:,0] < 0.0 and ca[:,1] < 0.0) else 1.0

    return s * np.sqrt(min(dot2(ca), dot2(cb)))

In [ ]:
def sdf_octahedron(p, s=0.7):
    p = np.abs(p)
    m = np.sum(p) - s

    if 3.0 * p[:,0] < m:
        q = p
    elif 3.0 * p[:,1] < m:
        q = np.array([p[:,1], p[:,2], p[:,0]])
    elif 3.0 * p[:,2] < m:
        q = np.array([p[:,2], p[:,0], p[:,1]])
    else:
        return m * 0.57735027

    k = np.clip(0.5 * (q[:,2] - q[:,1] + s), 0.0, s)
    return np.linalg.norm(np.array([q[:,0], q[:,1] - s + k, q[:,2] - k]))

In [ ]:
def sdf_vesica_segment(p, a=[0.7,0.7,0.7], b=[0.7,0.7,0.7], w=0.7):
    c = (a + b) * 0.5
    l = np.linalg.norm(b - a)
    v = (b - a) / l
    y = np.dot(p - c, v)
    q = np.array([np.linalg.norm(p - c - y * v), abs(y)])

    r = 0.5 * l
    d = 0.5 * (r * r - w * w) / w

    if r * q[:,0] < d * (q[:,1] - r):
        h = np.array([0.0, r, 0.0])
    else:
        h = np.array([-d, 0.0, d + w])

    return np.linalg.norm(q - h[:2]) - h[:,2]
